# Imports

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [3]:
gpd.read_file("../data/USA_Parks/v10/park_dtl.gdb/")

,NAME,FEATTYPE,MNFC,SQMI,Shape_Length,Shape_Area,geometry
0,Pago Pago Park,Local park,7170,0.02,0.010034,0.000005,"MULTIPOLYGON (((-170.70291 -14.27240, -170.703..."
1,National Park of American Samoa,National park or forest,7170,4.06,0.326497,0.000881,"MULTIPOLYGON (((-170.71479 -14.28528, -170.714..."
2,National Park of American Samoa,National park or forest,7170,8.36,0.313004,0.001814,"MULTIPOLYGON (((-169.48774 -14.27263, -169.487..."
3,Hawaiian Islands National Wildlife Refuge,Regional park,7170,2.67,0.327720,0.000620,"MULTIPOLYGON (((-161.92812 23.05704, -161.9281..."
4,Russian Fort Elizabeth State Historical Park,State park or forest,7170,0.03,0.010972,0.000006,"MULTIPOLYGON (((-159.66526 21.94916, -159.6652..."
...,...,...,...,...,...,...,...
59129,Two Lovers Point Park,Local park,7170,0.05,0.013617,0.000010,"MULTIPOLYGON (((144.80559 13.53381, 144.80564 ..."
59130,Anao Conservation Area,Regional park,7170,1.21,0.100135,0.000261,"MULTIPOLYGON (((144.92650 13.52184, 144.92646 ..."
59131,American Memorial Park,State park or forest,7170,0.22,0.069397,0.000048,"MULTIPOLYGON (((145.72624 15.21422, 145.72526 ..."
59132,Aleutian Islands World War II National Monument,National park or forest,7170,0.01,0.006503,0.000003,"MULTIPOLYGON (((173.18337 52.88374, 173.18306 ..."
